In [7]:
import nltk
from nltk import word_tokenize, sent_tokenize
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.manifold import TSNE
import pandas as pd
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
import jieba
import re
%matplotlib inline

In [31]:
training_path = 'training_with_tokens_jieba_with_stopwords.xlsx'
testing_path = 'testing_with_tokens_jieba_with_stopwords.xlsx'

df_train = pd.read_excel(training_path)
df_testing = pd.read_excel(testing_path)

In [3]:
df_train.head()

,class,description,tokens,jieba_tokens
0,2,合晟资产是一家专注于股票、债券等二级市场投资，为合格投资者提供专业资产管理服务的企业。公司业...,"['合晟资产', '是', '一家', '专注', '于', '股票', '、', '债券'...","['合晟', '资产', '是', '一家', '专注', '于', '股票', '、', ..."
1,2,公司的主营业务为向中小微企业、个体工商户、农户等客户提供贷款服务，自设立以来主营业务未发生过变化。,"['公司', '的', '主营', '业务', '为', '向', '中小', '微企业',...","['公司', '的', '主营业务', '为', '向', '中小', '微', '企业',..."
2,1,公司立足于商业地产服务，致力于为商业地产开发、销售、运营全产业链提供一整套增值服务，业务覆盖...,"['公司', '立足于', '商业地产', '服务', '，', '致力于', '为', '...","['公司', '立足于', '商业地产', '服务', '，', '致力于', '为', '..."
3,2,公司经工商管理部门核准的经营范围为“投资咨询、经济信息咨询，企业管理咨询，品牌推广策划，公共...,"['公司', '经', '工商管理部门', '核准', '的', '经营', '范围', '...","['公司', '经', '工商管理', '部门', '核准', '的', '经营范围', '..."
4,2,该公司的主营业务为在中国境内(港、澳、台除外)开展保险代理销售，依托于自身的产品研究能力和专...,"['该', '公司', '的', '主营', '业务', '为', '在', '中国', '...","['该', '公司', '的', '主营业务', '为', '在', '中国', '境内',..."


In [4]:
with open('./stop_words.txt', 'r') as f:
    lines = f.readlines()
    
stop_words = [x[:-1] for x in lines]

#### 从百度分词的结果中提取长度大于1的分词, 用来扩充jieba分词的词典

In [34]:
def restore_tokens_from_text(text):
    regex_to_list = r"[\[\',\]\(\)）（]"
    space_separated_string = re.sub(regex_to_list, '', text)
    return space_separated_string.split(' ')

def generate_dict_from_baidu_tokens(df, col_name):
    dict_tokens = set()
    for index, row in df.iterrows():
        tokens = restore_tokens_from_text(row[col_name])
        for token in tokens:
            if len(token) > 1:
                dict_tokens.add(token)
    return dict_tokens

In [35]:
words_from_baidu_training = generate_dict_from_baidu_tokens(df_train, col_name='tokens')
words_from_baidu_testing = generate_dict_from_baidu_tokens(df_testing, col_name='tokens')
words_from_baidu = list(words_from_baidu_training.union(words_from_baidu_testing))

In [36]:
words_from_baidu[:5]

['235.84元', '瞄准', '沁阳市', '界面', '以纯']

In [37]:
for word in words_from_baidu:
    jieba.add_word(word)

In [38]:
df_train.loc[0]['description']

'合晟资产是一家专注于股票、债券等二级市场投资，为合格投资者提供专业资产管理服务的企业。公司业务范围包括资产管理、投资咨询和投资顾问服务。公司管理的私募基金产品主要包括股票型、债券型资产管理计划或证券投资基金，管理总资产规模80亿元左右。根据中国证券投资基金业协会数据，公司管理的私募证券投资基金（顾问管理）类规模较大，公司管理规模处于50亿元以上的第一梯队。'

In [39]:
[x for x in jieba.lcut(df_train.loc[0]['description']) if x not in stop_words]

['合晟资产',
 '专注',
 '股票',
 '债券',
 '二级市场',
 '投资',
 '合格',
 '投资者',
 '资产',
 '管理',
 '企业',
 '业务范围',
 '资产',
 '管理',
 '投资',
 '咨询',
 '投资',
 '顾问',
 '管理',
 '私募基金',
 '股票型',
 '债券型',
 '资产',
 '管理',
 '计划',
 '证券投资',
 '基金',
 '管理',
 '总资产',
 '规模',
 '80亿元',
 '中国证券投资基金业协会',
 '数据',
 '管理',
 '私募证券投资基金',
 '顾问',
 '管理',
 '类',
 '规模',
 '较大',
 '管理',
 '规模',
 '处于',
 '50亿元',
 '第一',
 '梯队']

In [26]:
def save_df_to_excel(df, file_path):
    writer = pd.ExcelWriter(file_path)
    df.to_excel(writer, sheet_name='Sheet1')
    writer.save()

In [40]:
df_train['tokens_merged'] = df_train['description'].apply(lambda x: [word for word in jieba.lcut(x) if word not in stop_words])
df_testing['tokens_merged'] = df_testing['description'].apply(lambda x: [word for word in jieba.lcut(x) if word not in stop_words])

In [41]:
save_df_to_excel(df_train, 'training_with_tokens.xlsx')
save_df_to_excel(df_testing, 'testing_with_tokens.xlsx')